In [1]:
import os
import json
import random
import sys
import asyncio
import pickle
import datetime
sys.path.append('../')

from openai import OpenAI, AsyncClient
from json import JSONDecodeError
from tqdm.auto import tqdm
from utils import *
from pydantic import BaseModel

c:\Users\HAOXUAN\miniconda3\envs\or\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
cfg = json.load(open('../configs./configs.json', 'r'))
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])
asyncclient = AsyncClient(api_key=os.environ["OPENAI_API_KEY"])

dt = datetime.datetime.today().strftime('%Y-%m-%d-%H-%M-%S')

In [3]:
DATA_DIR = '../data'
DATASET_NAME = 'NL4OPT' 
OUTPUT_DIR = '../output'  

nl4opt_data = read_txt_file(os.path.join(DATA_DIR, DATASET_NAME, 'nl4opt.txt'))
questions, answers = get_nl4opt_qas(nl4opt_data)
assert len(questions) == len(answers)

qa_pairs = list(zip(questions, answers))
demo_samples, test_samples = get_demo_and_test_samples(qa_pairs)

questions = [q for q, _ in test_samples]
answers = [a for _, a in test_samples]

2024-09-17 12:28:25.065 | DEBUG    | utils:read_txt_file:14 - Reading file: ../data\NL4OPT\nl4opt.txt
2024-09-17 12:28:25.066 | DEBUG    | utils:read_txt_file:16 - File read successfully: ../data\NL4OPT\nl4opt.txt
2024-09-17 12:28:25.068 | INFO     | utils:get_nl4opt_qas:35 - Number of questions: 245
2024-09-17 12:28:25.068 | INFO     | utils:get_nl4opt_qas:36 - Number of answers: 245
2024-09-17 12:28:25.068 | INFO     | utils:get_demo_and_test_samples:47 - Number of demo samples: 20
2024-09-17 12:28:25.069 | INFO     | utils:get_demo_and_test_samples:48 - Number of test samples: 225


### GPT-4o-mini + Zero-Shot CoT + Structured Output = 21.33%

In [4]:
class Step(BaseModel):
    explanation: str
    output: str

class LPReasoning(BaseModel):
    steps: list[Step]
    final_answer: float

batch_size = 16
sys_prompt = """You are an expert in optimization problems. Your task is to find the optimal solution to the given problem. If there is no optimial solution, please return inf. Guide the user through the solution step by step."""

lp_reasoning_list = []
for idx in tqdm(range(0, len(questions), batch_size)):
    batch = questions[idx:idx+batch_size]
    
    tasks = [asyncclient.beta.chat.completions.parse(
        model="gpt-4o-mini",
        temperature=0,
        response_format=LPReasoning,
        messages=[
            {"role": "system", "content": sys_prompt},
            {"role": "user", "content": f"Problem: {q}"}
        ]) for q in batch
    ]

    combined_responses = await asyncio.gather(*tasks)
    lp_reasoning_list.extend([r.choices[0].message.parsed for r in combined_responses])

100%|██████████| 15/15 [03:16<00:00, 13.10s/it]


In [5]:
# use pickle to save the list
filename = 'lp_reasoning_list-' + dt + '.pkl'
with open(os.path.join(OUTPUT_DIR, filename), 'wb') as f:
    pickle.dump(lp_reasoning_list, f)

In [6]:
error_idx = []
pred_answers = []
all_pass = True
for i, lr in enumerate(lp_reasoning_list):
    try:
        pred_answers.append(lr.final_answer)
    except:
        all_pass = False
        error_idx.append(i)
        continue

if all_pass:
    loguru.logger.info(f"Successfully parsed all predictions.")

2024-09-17 12:31:59.038 | INFO     | __main__:<module>:13 - Successfully parsed all predictions.


In [7]:
assert len(pred_answers) == len(answers)

In [8]:
correct = []
for p, r in zip(pred_answers, answers):
    if (float(p) == np.inf and float(r) == np.inf) or (abs(float(p) - float(r)) / float(r) < 1e-2):
        correct.append(True)
    else:
        correct.append(False)

In [9]:
(sum(correct) / len(answers)) * 100

21.333333333333336